<a href="https://colab.research.google.com/github/Zarar-Azwar/AutoYOLOAnnotator-/blob/main/AutoYOLOAnnotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
import torch
import cv2
import os
from datetime import datetime
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import shutil
from typing import Dict, List, Union, Tuple
from pathlib import Path
from google.colab import files
%matplotlib inline

#### Florence 2 Import/Download

In [2]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

#### Inference

In [3]:
def inference(task_prompt,image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    print("Prompt: ",prompt)
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

#### Clips to Frames

In [4]:
class Clips_To_Frame:
  def __init__(self):
    pass

  @staticmethod
  def extract_frames(video_path, output_dir=None, frame_interval=1, output_format='jpg'):
      """
      Extract frames from a video file.

      Args:
          video_path (str): Path to the video file
          output_dir (str): Directory to save frames (default: creates 'frames_[timestamp]')
          frame_interval (int): Extract every nth frame (default: 1 = every frame)
          output_format (str): Format to save frames ('jpg' or 'png')

      Returns:
          str: Path to the output directory
      """
      # Input validation
      if not os.path.exists(video_path):
          raise FileNotFoundError(f"Video file not found: {video_path}")

      if output_format.lower() not in ['jpg', 'png']:
          raise ValueError("Output format must be 'jpg' or 'png'")

      # Create output directory if not specified
      if output_dir is None:
          timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
          output_dir = f'frames_{timestamp}'

      os.makedirs(output_dir, exist_ok=True)

      # Initialize video capture
      cap = cv2.VideoCapture(video_path)

      if not cap.isOpened():
          raise Exception("Error opening video file")

      # Get video properties
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      fps = int(cap.get(cv2.CAP_PROP_FPS))

      print(f"Video Properties:")
      print(f"Total Frames: {total_frames}")
      print(f"FPS: {fps}")
      print(f"Duration: {total_frames/fps:.2f} seconds")
      print(f"Extracting every {frame_interval} frame(s)")

      # Initialize variables
      frame_count = 0
      saved_count = 0

      # Process frames with progress bar
      with tqdm(total=total_frames, desc="Extracting frames") as pbar:
          while True:
              ret, frame = cap.read()

              if not ret:
                  break

              # Save frame if it meets the interval criteria
              if frame_count % frame_interval == 0:
                  # Generate frame filename
                  frame_filename = os.path.join(
                      output_dir,
                      f'frame_{frame_count:06d}.{output_format}'
                  )

                  # Save frame
                  if output_format.lower() == 'jpg':
                      cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                  else:  # png
                      cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_PNG_COMPRESSION, 9])

                  saved_count += 1

              frame_count += 1
              pbar.update(1)

      # Release resources
      cap.release()

      print(f"\nExtraction complete!")
      print(f"Saved {saved_count} frames to: {output_dir}")

      return output_dir
  @staticmethod
  def clip_to_frame_main(video_path):
      """
      Example usage of the frame extraction function
      """
      # Example configuration
      # video_path = "/content/chunk_006.mp4"  # Replace with your video path
      output_dir = "extracted_frames"  # Optional: specify output directory
      frame_interval = 30  # Extract one frame every second (assuming 30fps video)
      output_format = 'jpg'  # or 'png'
      ctf=Clips_To_Frame()
      try:
          # Extract frames
          output_path = ctf.extract_frames(
              video_path=video_path,
              output_dir=output_dir,
              frame_interval=frame_interval,
              output_format=output_format
          )

          print(f"Frames successfully extracted to: {output_path}")

      except Exception as e:
          print(f"Error: {str(e)}")


#### Plot Bounding Box on Extracted Frames

In [5]:
def plot_bbox(image, data):
   # Create a figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Plot each bounding box
    for bbox, label in zip(data['bboxes'], data['labels']):
        # Unpack the bounding box coordinates
        x1, y1, x2, y2 = bbox
        # Create a Rectangle patch
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
        # Add the rectangle to the Axes
        ax.add_patch(rect)
        # Annotate the label
        plt.text(x1, y1, label, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    # Remove the axis ticks and labels
    ax.axis('off')

    # Show the plot
    plt.show()

#### Creating Annotation (Format Florence Output to Yolo Annotation Format)

In [6]:


class YOLOAnnotationConverter:
    def __init__(self, output_dir: str = 'yolo_dataset'):
        self.output_dir = output_dir
        self.images_dir = os.path.join(output_dir, 'images')
        self.labels_dir = os.path.join(output_dir, 'labels')
        self.class_mapping = {}

        # Create directory structure
        os.makedirs(self.images_dir, exist_ok=True)
        os.makedirs(self.labels_dir, exist_ok=True)

    def create_class_mapping(self, labels: List[str]) -> Dict[str, int]:
        """Create class mapping from unique labels"""
        unique_labels = sorted(set(labels))
        self.class_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        return self.class_mapping

    def convert_to_yolo_format(
        self,
        annotations: Dict[str, Dict[str, Union[List[List[float]], List[str]]]],
        image_filename: str,
        image_width: int = 800,
        image_height: int = 600,
    ) -> str:
        """
        Convert detection annotations to YOLO format for a single image

        Args:
            annotations: Dictionary containing bboxes and labels
            image_filename: Original image filename
            image_width: Width of the image
            image_height: Height of the image

        Returns:
            Path to the saved annotation file
        """
        # Update class mapping if needed
        if not self.class_mapping:
            self.create_class_mapping(annotations['<OD>']['labels'])

        # Create YOLO format annotations
        yolo_annotations = []
        bboxes = annotations['<OD>']['bboxes']
        labels = annotations['<OD>']['labels']

        for bbox, label in zip(bboxes, labels):
            x_min, y_min, x_max, y_max = bbox

            # Calculate normalized center coordinates and dimensions
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height

            class_id = self.class_mapping[label]
            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_annotations.append(yolo_line)

        # Save annotations with same basename as image but .txt extension
        base_name = os.path.splitext(image_filename)[0]
        annotation_filename = f"{base_name}.txt"
        annotation_path = os.path.join(self.labels_dir, annotation_filename)

        with open(annotation_path, 'w') as f:
            f.write('\n'.join(yolo_annotations))

        return annotation_path

    def process_image_and_annotations(
        self,
        image_path: str,
        annotations: Dict,
        image_width: int = 800,
        image_height: int = 600
    ) -> Tuple[str, str]:
        """
        Process single image and its annotations

        Args:
            image_path: Path to the image file
            annotations: Annotations for the image
            image_width: Width of the image
            image_height: Height of the image

        Returns:
            Tuple of (image path, annotation path)
        """
        # Copy image to dataset directory
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(self.images_dir, image_filename)
        shutil.copy2(image_path, new_image_path)

        # Convert and save annotations
        annotation_path = self.convert_to_yolo_format(
            annotations,
            image_filename,
            image_width,
            image_height
        )

        return new_image_path, annotation_path

    def create_dataset_files(self):
        """Create classes.txt and dataset.yaml"""
        # Save classes.txt
        classes_path = os.path.join(self.output_dir, 'classes.txt')
        with open(classes_path, 'w') as f:
            for class_name in sorted(self.class_mapping, key=self.class_mapping.get):
                f.write(f"{class_name}\n")

        # Create dataset.yaml
        yaml_content = f"""
          path: {os.path.abspath(self.output_dir)}
          train: images  # train images
          val: images  # val images

          # Classes
          nc: {len(self.class_mapping)}  # number of classes
          names: {list(sorted(self.class_mapping, key=self.class_mapping.get))}  # class names
          """

        yaml_path = os.path.join(self.output_dir, 'dataset.yaml')
        with open(yaml_path, 'w') as f:
            f.write(yaml_content.strip())

def driver(image_annotations):
    # Example usage with multiple images
    converter = YOLOAnnotationConverter(output_dir='yolo_dataset')


    processed_files = []
    for image_filename, annotations in image_annotations.items():
        # In real usage, you would have actual image paths
        image_path = f"path/to/images/{image_filename}"

        # Skip processing if image doesn't exist (for this example)
        if not os.path.exists(image_path):
            print(f"Example: Would process {image_filename} if it existed")
            continue

        image_path, annotation_path = converter.process_image_and_annotations(
            image_path,
            annotations
        )
        processed_files.append((image_path, annotation_path))

    # Create dataset files
    converter.create_dataset_files()

    # Print summary
    print("\nDataset creation complete!")
    print(f"\nClass mapping:")
    for class_name, class_id in sorted(converter.class_mapping.items(), key=lambda x: x[1]):
        print(f"{class_name}: {class_id}")

    print("\nDirectory structure created:")
    print(f"├── {converter.output_dir}/")
    print(f"│   ├── images/")
    print(f"│   ├── labels/")
    print(f"│   ├── classes.txt")
    print(f"│   └── dataset.yaml")



#### Driver (Main Function)

In [ ]:


# Upload video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]  # Get the uploaded file name

# Process video frames
Clips_To_Frame.clip_to_frame_main(f"/content/{video_path}")

# Path for extracted frames
main_path = "/content/extracted_frames"
files = [os.path.join(main_path, file) for file in os.listdir(main_path)]

# Object Detection Inference
task_prompt = '<OD>'
for file in files:
    image = Image.open(file).convert('RGB')
    results = inference(task_prompt, image=image)
    driver(image_annotations=results)
    plot_bbox(image, results['<OD>'])

#### Output

1. Extracted Frames

   Frames extracted from video each frame is captured in 30 seconds

2. Yolo Dataset
   Directory structure created:  

```
├── yolo_dataset/
    │   ├── images/
    │   ├── labels/
    │   ├── classes.txt
    │   └── dataset.yaml
```





#### Downloading Frames and Yolo dataset

In [8]:
import shutil

frames_folder_path = "/content/extracted_frames"  # Change this to your folder path
frames_zip_path = "/content/extracted_frames.zip"

shutil.make_archive(frames_zip_path.replace('.zip', ''), 'zip', frames_folder_path)


dataset_folder_path = "/content/yolo_dataset"  # Change this to your folder path
dataset_zip_path = "/content/yolo_dataset.zip"

shutil.make_archive(dataset_zip_path.replace('.zip', ''), 'zip', dataset_folder_path)


'/content/yolo_dataset.zip'

In [9]:
from google.colab import files

files.download(frames_zip_path)
files.download(dataset_zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Deleting Folders (Optional)

In [10]:
!rm -rf /content/extracted_frames

In [11]:
!rm -rf /content/yolo_dataset